# Test pipeline

### Definições iniciais

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from scipy.linalg import sqrtm
from tqdm import trange
import random
random.seed(5)
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from datasets import lungCTData
from metrics import calculate_fid, FeatureExtractor, get_features
from metrics import my_ssim

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Dados

In [4]:
# Ajustar isso aqui para pasta com dados de teste
processed_data_folder = 'C:/Users/julia/OneDrive/Desktop/xuxu/IA376/projeto_final/dataset/Amostras'

In [5]:
dataset_test = lungCTData(processed_data_folder=processed_data_folder,mode='test')

In [6]:
np.shape(dataset_test)

(10, 2, 1, 512, 512)

# Calcula o FID

In [7]:
# Define a rede Inception V3
model = torch.hub.load('pytorch/vision:v0.10.0', 'inception_v3', pretrained=True)
model.eval()

# Define modelo para feature extraction
feature_extractor = FeatureExtractor(model)

Using cache found in C:\Users\julia/.cache\torch\hub\pytorch_vision_v0.10.0


In [8]:
# Obtém as features dos conjuntos de dados reais e dados sintéticos
features_real = get_features(feature_extractor, [0,1,2,3,4],dataset_test, choose_transform=2)
features_fake = get_features(feature_extractor, [5,6,7,8,9],dataset_test, choose_transform=2)

In [9]:
# Obtém a distribuições dos dados reais
mu1, sigma1 = np.mean(np.squeeze(features_real),axis=0), np.cov(np.squeeze(features_real))

# Calcula o FID dentro de uma mesma distribuição (só para testar se dá zero mesmo)
fid_value = calculate_fid(mu1, sigma1, mu1, sigma1)
print(f"FID: {fid_value:.2f}")

FID: 0.00


In [10]:
# Obtém as distribuições para os dados reais e sintéticos
mu1, sigma1 = np.mean(np.squeeze(features_real),axis=0), np.cov(np.squeeze(features_real))
mu2, sigma2 = np.mean(np.squeeze(features_fake),axis=0), np.cov(np.squeeze(features_fake))

# Calcula o FID entre os dados reais e sintéticos
fid_value = calculate_fid(mu1, sigma1, mu2, sigma2)
print(f"FID: {fid_value:.2f}")

FID: 24.98


# Calcula o SSIM

In [11]:
# Obtém os vetores de dados reais e sintéticos

dataset_real=[]
dataset_fake=[]
for i in range(5):
    ct, lung = dataset_test.__getitem__(i)
    dataset_real.append(ct.detach().numpy())

    ct, lung = dataset_test.__getitem__(i+5)
    dataset_fake.append(ct.detach().numpy())

np.shape(np.squeeze(dataset_real[0]))

(512, 512)

In [12]:
# Calcula SSIM entre dados reais e reais (verifica que saída é igual a 1 para entradas iguais)
ssim = np.zeros(5)
for i in range(5):
    ssim[i] = my_ssim(np.squeeze(dataset_real[i]), np.squeeze(dataset_real[i]), 0.01, 0.03, 0.03)

print(np.mean(ssim), np.std(ssim))

luminance: 1.00
contrast: 1.00
structure_similarity: 1.00
luminance: 1.00
contrast: 1.00
structure_similarity: 1.00
luminance: 1.00
contrast: 1.00
structure_similarity: 1.00
luminance: 1.00
contrast: 1.00
structure_similarity: 1.00
luminance: 1.00
contrast: 1.00
structure_similarity: 1.00
1.0000000029998395 3.234989966031e-08


In [13]:
# Calcula SSIM entre dados reais e sintéticos
ssim = np.zeros(5)
for i in range(5):
    ssim[i] = my_ssim(np.squeeze(dataset_real[i]), np.squeeze(dataset_fake[i]), 0.01, 0.03, 0.03)

print(np.mean(ssim), np.std(ssim))

luminance: 1.00
contrast: 1.00
structure_similarity: 0.99
luminance: 1.00
contrast: 1.00
structure_similarity: 0.99
luminance: 1.00
contrast: 1.00
structure_similarity: 0.99
luminance: 1.00
contrast: 1.00
structure_similarity: 0.99
luminance: 1.00
contrast: 1.00
structure_similarity: 0.99
0.9925000689677214 0.00016969069869950275


# 